In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, max as spark_max

In [2]:
# Initialize Spark Session
spark = SparkSession.builder.appName("EmployeesAssignment").getOrCreate()

In [3]:
# Load the CSV file into a Spark DataFrame
df = spark.read.csv("employees(in).csv", header=True, inferSchema=True)

In [4]:
# Show the DataFrame
df.show(5)

+-----------+-------+----------+------+------------+-------------+
|employee_id|   name|department|salary|joining_year|bonus_percent|
+-----------+-------+----------+------+------------+-------------+
|        101|  Alice|        HR| 50000|        2018|            5|
|        102|    Bob|   Finance| 60000|        2019|            7|
|        103|Charlie|        IT| 75000|        2017|           10|
|        104|  David|   Finance| 62000|        2020|            6|
|        105|    Eva|        IT| 80000|        2021|           12|
+-----------+-------+----------+------+------------+-------------+
only showing top 5 rows



In [5]:
# Select only the name and salary columns
df.select("name", "salary").show()

+-------+------+
|   name|salary|
+-------+------+
|  Alice| 50000|
|    Bob| 60000|
|Charlie| 75000|
|  David| 62000|
|    Eva| 80000|
|  Frank| 52000|
|  Grace| 90000|
+-------+------+



In [6]:
#  Filter employees with salary greater than 60000
df.filter(col("salary") > 60000).show()

+-----------+-------+----------+------+------------+-------------+
|employee_id|   name|department|salary|joining_year|bonus_percent|
+-----------+-------+----------+------+------------+-------------+
|        103|Charlie|        IT| 75000|        2017|           10|
|        104|  David|   Finance| 62000|        2020|            6|
|        105|    Eva|        IT| 80000|        2021|           12|
|        107|  Grace|        IT| 90000|        2019|           15|
+-----------+-------+----------+------+------------+-------------+



In [7]:
#  Add a new column bonus_amount = salary * bonus_percent / 100
df = df.withColumn("bonus_amount", col("salary") * col("bonus_percent") / 100)
df.show(5)

+-----------+-------+----------+------+------------+-------------+------------+
|employee_id|   name|department|salary|joining_year|bonus_percent|bonus_amount|
+-----------+-------+----------+------+------------+-------------+------------+
|        101|  Alice|        HR| 50000|        2018|            5|      2500.0|
|        102|    Bob|   Finance| 60000|        2019|            7|      4200.0|
|        103|Charlie|        IT| 75000|        2017|           10|      7500.0|
|        104|  David|   Finance| 62000|        2020|            6|      3720.0|
|        105|    Eva|        IT| 80000|        2021|           12|      9600.0|
+-----------+-------+----------+------+------------+-------------+------------+
only showing top 5 rows



In [8]:
#  Rename the column joining_year to year_joined
df = df.withColumnRenamed("joining_year", "year_joined")
df.show(5)

+-----------+-------+----------+------+-----------+-------------+------------+
|employee_id|   name|department|salary|year_joined|bonus_percent|bonus_amount|
+-----------+-------+----------+------+-----------+-------------+------------+
|        101|  Alice|        HR| 50000|       2018|            5|      2500.0|
|        102|    Bob|   Finance| 60000|       2019|            7|      4200.0|
|        103|Charlie|        IT| 75000|       2017|           10|      7500.0|
|        104|  David|   Finance| 62000|       2020|            6|      3720.0|
|        105|    Eva|        IT| 80000|       2021|           12|      9600.0|
+-----------+-------+----------+------+-----------+-------------+------------+
only showing top 5 rows



In [9]:
#  Drop the bonus_percent column
df = df.drop("bonus_percent")
df.show(5)

+-----------+-------+----------+------+-----------+------------+
|employee_id|   name|department|salary|year_joined|bonus_amount|
+-----------+-------+----------+------+-----------+------------+
|        101|  Alice|        HR| 50000|       2018|      2500.0|
|        102|    Bob|   Finance| 60000|       2019|      4200.0|
|        103|Charlie|        IT| 75000|       2017|      7500.0|
|        104|  David|   Finance| 62000|       2020|      3720.0|
|        105|    Eva|        IT| 80000|       2021|      9600.0|
+-----------+-------+----------+------+-----------+------------+
only showing top 5 rows



In [10]:
#  Count the number of employees in each department
df.groupBy("department").agg(avg("salary").alias("avg_salary")).show()

+----------+-----------------+
|department|       avg_salary|
+----------+-----------------+
|        HR|          51000.0|
|   Finance|          61000.0|
|        IT|81666.66666666667|
+----------+-----------------+



In [11]:
#  Find the maximum salary in the dataset
df.groupBy("department").count().show()

+----------+-----+
|department|count|
+----------+-----+
|        HR|    2|
|   Finance|    2|
|        IT|    3|
+----------+-----+



In [12]:
#  Sort employees by salary in descending order
df.agg(spark_max("salary").alias("max_salary")).show()

+----------+
|max_salary|
+----------+
|     90000|
+----------+



In [13]:
#  Sort employees by joining year in ascending order
df.orderBy(col("salary").desc()).show(5)

+-----------+-------+----------+------+-----------+------------+
|employee_id|   name|department|salary|year_joined|bonus_amount|
+-----------+-------+----------+------+-----------+------------+
|        107|  Grace|        IT| 90000|       2019|     13500.0|
|        105|    Eva|        IT| 80000|       2021|      9600.0|
|        103|Charlie|        IT| 75000|       2017|      7500.0|
|        104|  David|   Finance| 62000|       2020|      3720.0|
|        102|    Bob|   Finance| 60000|       2019|      4200.0|
+-----------+-------+----------+------+-----------+------------+
only showing top 5 rows



In [14]:
#  Create another DataFrame with department details
df.orderBy(col("year_joined").asc()).show(5)

+-----------+-------+----------+------+-----------+------------+
|employee_id|   name|department|salary|year_joined|bonus_amount|
+-----------+-------+----------+------+-----------+------------+
|        103|Charlie|        IT| 75000|       2017|      7500.0|
|        101|  Alice|        HR| 50000|       2018|      2500.0|
|        106|  Frank|        HR| 52000|       2018|      2600.0|
|        102|    Bob|   Finance| 60000|       2019|      4200.0|
|        107|  Grace|        IT| 90000|       2019|     13500.0|
+-----------+-------+----------+------+-----------+------------+
only showing top 5 rows



In [15]:
dept_data = [
    ("HR", "New York"),
    ("IT", "San Francisco"),
    ("Finance", "Chicago"),
    ("Sales", "Boston")
]

In [16]:
dept_columns = ["dept_name", "location"]

In [17]:
dept_df = spark.createDataFrame(dept_data, dept_columns)

In [18]:
# Join with employees DataFrame on department
joined_df = df.join(dept_df, df.department == dept_df.dept_name, "inner")


+-----------+-------+----------+------+-----------+------------+---------+-------------+
|employee_id|   name|department|salary|year_joined|bonus_amount|dept_name|     location|
+-----------+-------+----------+------+-----------+------------+---------+-------------+
|        106|  Frank|        HR| 52000|       2018|      2600.0|       HR|     New York|
|        101|  Alice|        HR| 50000|       2018|      2500.0|       HR|     New York|
|        107|  Grace|        IT| 90000|       2019|     13500.0|       IT|San Francisco|
|        105|    Eva|        IT| 80000|       2021|      9600.0|       IT|San Francisco|
|        103|Charlie|        IT| 75000|       2017|      7500.0|       IT|San Francisco|
+-----------+-------+----------+------+-----------+------------+---------+-------------+
only showing top 5 rows



In [27]:
#Display the first 5 rows using show
joined_df.show(5)

+-----------+-------+----------+------+-----------+------------+---------+-------------+
|employee_id|   name|department|salary|year_joined|bonus_amount|dept_name|     location|
+-----------+-------+----------+------+-----------+------------+---------+-------------+
|        106|  Frank|        HR| 52000|       2018|      2600.0|       HR|     New York|
|        101|  Alice|        HR| 50000|       2018|      2500.0|       HR|     New York|
|        107|  Grace|        IT| 90000|       2019|     13500.0|       IT|San Francisco|
|        105|    Eva|        IT| 80000|       2021|      9600.0|       IT|San Francisco|
|        103|Charlie|        IT| 75000|       2017|      7500.0|       IT|San Francisco|
+-----------+-------+----------+------+-----------+------------+---------+-------------+
only showing top 5 rows



In [20]:
#Find the total number of employees using count
print("Total Employees:", df.count())

Total Employees: 7


In [21]:
#Collect all rows into Python objects using collect
all_rows = df.collect()
print(all_rows[:3])  #

[Row(employee_id=101, name='Alice', department='HR', salary=50000, year_joined=2018, bonus_amount=2500.0), Row(employee_id=102, name='Bob', department='Finance', salary=60000, year_joined=2019, bonus_amount=4200.0), Row(employee_id=103, name='Charlie', department='IT', salary=75000, year_joined=2017, bonus_amount=7500.0)]


In [24]:
# Step 18:
first_three = df.take(3)
print(first_three)

[Row(employee_id=101, name='Alice', department='HR', salary=50000, year_joined=2018, bonus_amount=2500.0), Row(employee_id=102, name='Bob', department='Finance', salary=60000, year_joined=2019, bonus_amount=4200.0), Row(employee_id=103, name='Charlie', department='IT', salary=75000, year_joined=2017, bonus_amount=7500.0)]
